# SDK Examples - KPIs

Examples for new KPI functions that were released with smsdk v1.1

*created April 2023*

In [43]:
from smsdk import client
from datetime import datetime, timedelta
import pandas as pd

In [44]:
api_key = ''
api_secret = ''
tenant = 'demo-continuous'

cli = client.Client(tenant)
cli.login('apikey', 
          key_id = api_key, 
          secret_id = api_secret)

machine_types = cli.get_machine_type_names()
machine_types

['Downtime',
 'Paper_Mill_Finishing_and_Shipping',
 'Paper_Mill_PM1_Broke',
 'Paper_Mill_PM1_Calender_Stacks',
 'Paper_Mill_PM1_Dryer_Section',
 'Paper_Mill_PM1_Forming_Section',
 'Paper_Mill_PM1_Headbox',
 'Paper_Mill_PM1_Lab_Tests',
 'Paper_Mill_PM1_Press_Section',
 'Paper_Mill_PM1_Production_Status',
 'Paper_Mill_PM1_Reel',
 'Paper_Mill_PM1_Scale',
 'Paper_Mill_PM1_Steam_System',
 'Paper_Mill_PM1_Stock_Approach',
 'Paper_Mill_PM1_White_Water',
 'Paper_Mill_PM2_Broke',
 'Paper_Mill_PM2_Calendar',
 'Paper_Mill_PM2_Dryer_Section',
 'Paper_Mill_PM2_Forming_Section',
 'Paper_Mill_PM2_Headbox',
 'Paper_Mill_PM2_Lab_Tests',
 'Paper_Mill_PM2_Press_Section',
 'Paper_Mill_PM2_Production_Status',
 'Paper_Mill_PM2_Reel',
 'Paper_Mill_PM2_Scale',
 'Paper_Mill_PM2_Steam_System',
 'Paper_Mill_PM2_Stock_Approach',
 'Paper_Mill_PM2_White_Water',
 'Pulp_Mill_Num1_OCC_Coarse_Screen',
 'Pulp_Mill_Num1_OCC_Disc_Thickener',
 'Pulp_Mill_Num1_OCC_Fine_Screen',
 'Pulp_Mill_Num1_OCC_HD_Cleaning',
 'Pulp_Mill

# KPIs
---

The SDK has three functions related to KPIs. The first returns a list of all availible KPIs. The second of which allows a user to see which KPIs are availible for a particular asset. The third makes use of our Data Visualization API which allows a user to see these KPIs over a timeframe.

## Get KPIs

```cli.get_kpis()```

List the available KPIs for all assets.

In [45]:
kpis_dict = cli.get_kpis()
kpis_dict

[{'name': 'quality',
  'display_name': 'Quality',
  'formula': 'good_tons / (good_tons + (reject_tons + (random * 100))) * 100 if (good_tons + (reject_tons + (random * 100))) > 0 else None',
  'data_type': '',
  'dependencies': [{'aggregate': 'sum', 'name': 'reject_tons'},
   {'aggregate': 'sum', 'name': 'good_tons'},
   {'aggregate': 'sum', 'name': 'random'}]},
 {'name': 'performance',
  'display_name': 'Performance',
  'formula': '(machine_speed / max_speed) * 100 if max_speed > 0 else None',
  'data_type': '',
  'dependencies': [{'aggregate': 'sum', 'name': 'machine_speed'},
   {'aggregate': 'sum', 'name': 'max_speed'}]},
 {'name': 'availability',
  'display_name': 'Availability',
  'formula': '(availability / denominator) * 100 if denominator > 0 else None',
  'data_type': '',
  'dependencies': [{'aggregate': 'sum', 'name': 'availability'},
   {'aggregate': 'sum', 'name': 'denominator'}]},
 {'name': 'oee',
  'display_name': 'OEE',
  'formula': '(good_tons / (good_tons + (reject_ton

In [46]:
# example: create a table with KPIs and formulas
df_kpis = pd.DataFrame(kpis_dict)
df_kpis

,name,display_name,formula,data_type,dependencies
0,quality,Quality,good_tons / (good_tons + (reject_tons + (rando...,,"[{'aggregate': 'sum', 'name': 'reject_tons'}, ..."
1,performance,Performance,(machine_speed / max_speed) * 100 if max_speed...,,"[{'aggregate': 'sum', 'name': 'machine_speed'}..."
2,availability,Availability,(availability / denominator) * 100 if denomina...,,"[{'aggregate': 'sum', 'name': 'availability'},..."
3,oee,OEE,(good_tons / (good_tons + (reject_tons + (rand...,,"[{'aggregate': 'sum', 'name': 'reject_tons'}, ..."


## Get KPIs for Asset

```cli.get_kpis_for_asset(**asset_selection)```

List the available KPIs for a specific asset. For asset selection, see the example below or [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/asset_selection.md) GitHub doc.

**NOTE: machine type names must be the raw ones, not the clean/display names printed above!!!**

In [47]:
# get the raw machine type names (requires a sort of roundabout method)
def get_raw_machine_type(clean_machine_type: str) -> str:
    machines = cli.get_machine_names(source_type=clean_machine_type)
    return(cli.get_type_from_machine(machines[0]))

In [48]:
# KPIs by machine type
machine_types_clean = ['Paper_Mill_PM1_Production_Status', 'Paper_Mill_PM2_Production_Status']
asset1 = { 
    "asset_selection": {
        'machine_type': [get_raw_machine_type(i) for i in machine_types_clean]
    }
}
kpis_asset1 = cli.get_kpis_for_asset(**asset1)
pd.DataFrame(kpis_asset1)

,name,display_name,unit,type,data_type,stream_types,raw_data_field
0,performance,Performance,,continuous,float,[],
1,oee,OEE,,continuous,float,[],
2,quality,Quality,,continuous,float,[],
3,availability,Availability,,continuous,float,[],


In [49]:
# check machines of some type
prod_status_machines = cli.get_machine_names(source_type='Paper_Mill_PM1_Production_Status')


# KPIs by machine
asset2 = {
    'asset_selection': {
        'machine_type': [get_raw_machine_type("Paper_Mill_PM1_Production_Status")],
        'machine_source': prod_status_machines[0]
    }
}
kpis_asset2 = cli.get_kpis_for_asset(**asset2)
pd.DataFrame(kpis_asset2)

,name,display_name,unit,type,data_type,stream_types,raw_data_field
0,performance,Performance,,continuous,float,[],
1,oee,OEE,,continuous,float,[],
2,quality,Quality,,continuous,float,[],
3,availability,Availability,,continuous,float,[],


## Get KPIs over Time

```cli.get_kpi_data_viz(machine_source, kpis, i_vars, time_selection, **optional_data_viz_query)```

Get the value of a chosen KPI over some specific period of time. See examples of how to use all the parameters below. Find additional info about parameter formatting on [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/data_viz_query.md) GitHub doc.

```machine_source```: a list of machine names

In [50]:
this_machine_sources = [prod_status_machines[0]]

```kpis```: a list of names of kpis that you want data for

In [51]:
# query all KPIs - it should return only those available for this machine
this_kpis = df_kpis["name"].to_list()

```i_vars```: a list of dicts, one per independent (x-axis) variable -- for our purposes, usually a list of size one, where the one dict has details about some time field

In [52]:
this_i_vars = [
    {
      "name": "endtime",  # name of independent variable
      "time_resolution": "day",   # choose anything from 'year' down to 'second'
    #   "query_tz": "America/Los_Angeles",    # optional - timezone that query is in
    #   "output_tz": "America/Los_Angeles",  # optional - timezone that returned data should be in
    #   "bin_strategy": "user_defined2",  # optional - how to bin the data ('user_defined2', 'none', or 'categorical')
    #   "bin_count": 50   # optional - how many bins you want the data in
    }
]

```time_selection```: a dictionary for time selection, with all the same options as MA Data Viz (see [this](https://github.com/sightmachine/sightmachine-sdk/blob/master/docs/commonly_used_data_types/data_viz_query.md#time_selection) GitHub doc for more specifics)

In [53]:
# relative time selection
this_time_selection = {
    "time_type": "relative",
    "relative_start": 7,
    "relative_unit": "year",
    "ctime_tz": "America/Los_Angeles"
}

# absolution time selection
# this_time_selection = {
#   "time_type": "absolute",
#   "start_time": "2023-02-23T08:00:00.000Z",
#   "end_time": "2023-03-01T21:35:35.499Z",
#   "time_zone": "America/Los_Angeles"
# }

This function may take a minute to run. If you run into issues, make sure the KPI you're looking for exists for the asset you're looking for using get_kpis_for_asset (above). Also, try creating the same query in MA. If it works, use the Developer Panel of your browser to view the contents of the API call.

In [54]:
cli.get_kpi_data_viz(machine_sources=this_machine_sources,
                     kpis=this_kpis, i_vars=this_i_vars,
                     time_selection=this_time_selection)

[{'i_vals': {'endtime': {'i_pos': 0,
    'bin_no': 0,
    'bin_min': '2022-07-01T00:00:00+00:00',
    'bin_max': '2022-07-01T00:00:00+00:00',
    'bin_avg': '2022-07-01T00:00:00+00:00'}},
  'd_vals': {'quality': {'avg': 90.7835979686744},
   'performance': {'avg': 89.49735351240147},
   'availability': {'avg': 89.46474470302188},
   'oee': {'avg': 72.6891367095519}},
  '_count': 2879,
  'kpi_dependencies': {'quality': {'reject_tons': 0.0,
    'good_tons': 1387960.6101074219,
    'random': 1409.0654339132886},
   'performance': {'machine_speed': 7980972.46408236,
    'max_speed': 8917551.358629227},
   'availability': {'availability': 257569.0, 'denominator': 287900.0},
   'oee': {'reject_tons': 0.0,
    'good_tons': 1387960.6101074219,
    'machine_speed': 7980972.46408236,
    'max_speed': 8917551.358629227,
    'availability': 257569.0,
    'denominator': 287900.0,
    'random': 1409.0654339132886}}},
 {'i_vals': {'endtime': {'i_pos': 0,
    'bin_no': 1,
    'bin_min': '2022-07-02T00

Using additional args - Where

In [55]:
this_args = {'where': [{
    "name": "shift",
    "op": "eq",
    "value": "Crew B"
}]
}

data_dict = cli.get_kpi_data_viz(machine_sources=this_machine_sources,
                     kpis=this_kpis, i_vars=this_i_vars,
                     time_selection=this_time_selection,
                     **this_args)
data_dict[0]

{'i_vals': {'endtime': {'i_pos': 0,
   'bin_no': 0,
   'bin_min': '2022-07-01T00:00:00+00:00',
   'bin_max': '2022-07-01T00:00:00+00:00',
   'bin_avg': '2022-07-01T00:00:00+00:00'}},
 'd_vals': {'quality': {'avg': 91.04619889128857},
  'performance': {'avg': 88.7779862177258},
  'availability': {'avg': 88.78242677824268},
  'oee': {'avg': 71.7619316740485}},
 '_count': 239,
 'kpi_dependencies': {'quality': {'reject_tons': 0.0,
   'good_tons': 122937.05456542969,
   'random': 120.90059210313545},
  'performance': {'machine_speed': 701269.1557617188,
   'max_speed': 789913.3395996094},
  'availability': {'availability': 21219.0, 'denominator': 23900.0},
  'oee': {'reject_tons': 0.0,
   'good_tons': 122937.05456542969,
   'machine_speed': 701269.1557617188,
   'max_speed': 789913.3395996094,
   'availability': 21219.0,
   'denominator': 23900.0,
   'random': 120.90059210313545}}}

In [56]:
# example: extract a table
data_list = []
for point in data_dict:
    new_row = {}
    # extract i_vals (assuming it's just one - time)
    new_row["endtime"] = point["i_vals"]["endtime"]["bin_avg"]
    # extract d_vals
    for d in point["d_vals"].keys():
        new_row[d] = point["d_vals"][d]["avg"]
    # extract other (dependent) values
    for kpi in point["kpi_dependencies"].keys():
        for field in point["kpi_dependencies"][kpi].keys():
            new_row[field] = point["kpi_dependencies"][kpi][field]
    # add to list of rows
    data_list.append(new_row)
df_kpis = pd.DataFrame(data_list)
        
# now you can graph the data using this table

df_kpis.head()

,endtime,quality,performance,availability,oee,reject_tons,good_tons,random,machine_speed,max_speed,denominator
0,2022-07-01T00:00:00+00:00,91.046199,88.777986,21219.0,71.761932,0.000000,1.229371e+05,120.900592,7.012692e+05,7.899133e+05,23900.0
1,2022-07-02T00:00:00+00:00,91.097723,89.052513,128466.0,72.373473,0.000000,7.392190e+05,722.381719,3.899135e+06,4.378467e+06,144000.0
2,2022-07-03T00:00:00+00:00,91.512301,88.637716,127724.0,71.946231,29.271001,7.749197e+05,718.439657,3.297092e+06,3.719739e+06,144000.0
3,2022-07-04T00:00:00+00:00,93.290032,89.719689,129369.0,75.195306,61.870001,1.018301e+06,731.803363,2.959415e+06,3.298512e+06,144000.0
4,2022-07-05T00:00:00+00:00,94.105393,89.741744,107915.0,75.883583,597.150010,9.598014e+05,595.232315,2.781879e+06,3.099872e+06,120100.0
